In [3]:
import pandas as pd
import numpy as np

In [4]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')


In [8]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixFactv1.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9', 'test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixFactv1.csv
day8_features_matrixFactv1.csv
day9_features_matrixFactv1.csv
test_features_matrixFactv1.csv


In [40]:
import gc
del train
del test
del df_dict
gc.collect()

1688

# Get Compensate Group

Get those only appears on test

In [5]:
def intersec_category_test(df_train_all, df_test_all,  col_list):
    missing = {}
    for col in col_list:
        print('processing: {}'.format(col))
        train_index = set(df_train_all[col].value_counts().index)
        test_index = set(df_test_all[col].value_counts().index)                 
        inter_index = list(train_index.intersection(test_index))
        print('whole train index length is: {}'.format(len(train_index)))
        print('test index length is: {}'.format(len(test_index)))
        print('intersection index length is: {}'.format(len(inter_index)))
        miss_value = list(test_index - set(inter_index))
        print('missing length: {}'.format(len(miss_value)))
        missing[col] = list(miss_value)
        print('---')
    return missing
col_list = [ 'app', 'device', 'os', 'channel']
tmp_test = intersec_category_test(train.copy(), test.copy(), col_list)

processing: app
whole train index length is: 706
test index length is: 417
intersection index length is: 393
missing length: 24
---
processing: device
whole train index length is: 3475
test index length is: 1985
intersection index length is: 1661
missing length: 324
---
processing: os
whole train index length is: 800
test index length is: 395
intersection index length is: 339
missing length: 56
---
processing: channel
whole train index length is: 202
test index length is: 178
intersection index length is: 178
missing length: 0
---


## get those appear on train but not in day8 day9 day10

In [15]:
def intersec_category(df_train_all, df_test_all, df, col_list):
    missing = {}
    for col in col_list:
        print('processing: {}'.format(col))
        train_index = set(df_train_all[col].value_counts().index)
        test_index = set(df_test_all[col].value_counts().index)                 
        inter_index = list(train_index.intersection(test_index))
        print('whole train index length is: {}'.format(len(train_index)))
        print('test index length is: {}'.format(len(test_index)))
        print('intersection index length is: {}'.format(len(inter_index)))
        miss_value = set(inter_index) - set(df[col].value_counts().index)
        print('missing length: {}'.format(len(miss_value)))
        missing[col] = list(miss_value)
        print('---')
    return missing
col_list = [ 'app', 'device', 'os', 'channel']
tmp = intersec_category(train.copy(), test.copy(), pd.concat([df_dict['day7'],df_dict['day8'],df_dict['day9']]), col_list)

processing: app
whole train index length is: 706
test index length is: 417
intersection index length is: 393
missing length: 21
---
processing: device
whole train index length is: 3475
test index length is: 1985
intersection index length is: 1661
missing length: 224
---
processing: os
whole train index length is: 800
test index length is: 395
intersection index length is: 339
missing length: 39
---
processing: channel
whole train index length is: 202
test index length is: 178
intersection index length is: 178
missing length: 0
---


In [17]:
history_index = set(train.index) - set(df_dict['day7'].index) -set(df_dict['day8'].index) -set(df_dict['day9'].index)

In [19]:
df_history = train.iloc[list(history_index)]

In [6]:
import multiprocessing
import pandas as pd
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    col = kwargs.pop('col')
    diction = kwargs.pop('value_dict')
    return df[col].apply(func, args=(col,diction,))

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
    
def square(x):
    return x**x

def func(x, col, diction):
    return x in diction[col]



### deal with app

In [7]:
worker = 10
# app = apply_by_multiprocessing(df_history, func, axis=1, workers=worker, col='app', value_dict=tmp)  
app_test = apply_by_multiprocessing(test, func, axis=1, workers=worker, col='app', value_dict=tmp_test)  


In [8]:
# app = df_history[app]
app_test = test[app_test]


In [94]:
# len(app)
len(app_test)

4436

In [9]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/compensate/'
np.save(load_path+'app_onlyappearon_test.npy', app_test.index.values)

### deal with device

In [10]:
# device = apply_by_multiprocessing(df_history, func, axis=1, workers=worker, col='device', value_dict=tmp)  
device_test = apply_by_multiprocessing(test, func, axis=1, workers=worker, col='device', value_dict=tmp_test)  

In [11]:
# device = df_history[device]
device_test = test[device_test]

In [97]:
# len(device)
len(device_test)

77196

In [12]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/compensate/'
np.save(load_path+'device_onlyappearon_test.npy', device_test.index.values)

### deal with os

In [13]:
# os = apply_by_multiprocessing(df_history, func, axis=1, workers=worker, col='os', value_dict=tmp)  
os_test = apply_by_multiprocessing(test, func, axis=1, workers=worker, col='os', value_dict=tmp_test)  

In [14]:
# os = df_history[os]
os_test = test[os_test]

In [15]:
# len(os)
len(os_test)

128743

In [16]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/compensate/'
np.save(load_path+'os_onlyappearon_test.npy', os_test.index.values)

# Analysis

In [106]:
all_comp = list(set(list(app_test.index.values) + list(device_test.index.values) + list(os_test.index.values)))

In [107]:
len(all_comp)

130073

In [65]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/compensate/'
np.save(load_path+'all_compen.npy', np.array(all_comp))

In [67]:
df_comp = df_history.loc[all_comp]

In [71]:
len(df_comp[df_comp.is_attributed == 0])/len(df_comp)

0.8914634146341464

In [85]:
18000000*(1-0.9925)

134999.99999999913

# evaluate

In [14]:
def intersec_category(df_train_all, df_test_all, df, col_list):
    for col in col_list:
        print('processing: {}'.format(col))
        train_index = set(df_train_all[col].value_counts().index)
        test_index = set(df_test_all[col].value_counts().index)                 
        inter_index = list(train_index.intersection(test_index))
        print('train index length is: {}'.format(len(train_index)))
        print('test index length is: {}'.format(len(test_index)))
        print('intersection index length is: {}'.format(len(inter_index)))
        index_map = pd.Series(inter_index, index=inter_index)
        df.loc[:,col] = df[col].map(index_map).fillna(-1)
        print('---')
    return df
col_list = [ 'app', 'device', 'os', 'channel']
tmp = intersec_category(pd.concat([df_dict['day7'],df_dict['day8'],df_dict['day9']]), test.copy(), test.copy(),col_list)
    

processing: app
train index length is: 556
test index length is: 417
intersection index length is: 372
---
processing: device
train index length is: 2438
test index length is: 1985
intersection index length is: 1437
---
processing: os
train index length is: 492
test index length is: 395
intersection index length is: 300
---
processing: channel
train index length is: 193
test index length is: 178
intersection index length is: 178
---


In [3]:
### channel app os

In [4]:
day = train['day'].value_counts()

In [10]:
group = train.groupby(['day'])

In [54]:
# val - front
total_length = len(train)
val_length = 1600 * 10000
train_length = 5900 * 10000

valset = {}
trainset = {}
lgb_val = train.iloc[total_length-(val_length+train_length): total_length-train_length]
lgb_train = train.iloc[total_length-train_length: ]

for col in ['channel', 'app', 'os', 'device', 'ip']:
    valset[col] = set(lgb_val[col].value_counts().index.values)
#     valset[col] = set(train.loc[total_length-75000000: total_length-52500000,col].value_counts().index.values)
    trainset[col] = set(lgb_train[col].value_counts().index.values)

In [38]:
# val - rear
total_length = len(train)
val_length = 2000 * 10000
train_length = 5500 * 10000

valset = {}
trainset = {}
lgb_val = train.iloc[total_length-val_length:]
lgb_train = train.iloc[total_length-(val_length+train_length): total_length-val_length]

for col in ['channel', 'app', 'os', 'device']:
    valset[col] = set(lgb_val[col].value_counts().index.values)
#     valset[col] = set(train.loc[total_length-75000000: total_length-52500000,col].value_counts().index.values)
    trainset[col] = set(lgb_train[col].value_counts().index.values)

In [39]:
#### val
total_length = len(train)
val_length = 2150 * 10000
train_length = 5500 * 10000
val_front = 1
val_front_length = int(val_length * val_front)
val_rear_length = val_length - val_front_length
valset = {}
trainset = {}

lgb_val_front = train.iloc[total_length-(val_length+train_length): total_length-(val_length+train_length) + val_front_length]
print(len(lgb_val_front))
lgb_val_rear = train.iloc[total_length-(val_rear_length):]
print(len(lgb_val_rear))
lgb_val = pd.concat([lgb_val_front,lgb_val_rear])
lgb_train = train.iloc[total_length-(val_length+train_length) + val_front_length: total_length-(val_rear_length)]

print(len(lgb_val))
print(len(lgb_train))

for col in ['channel', 'app', 'os', 'device']:
    trainset[col] = set(lgb_train[col].value_counts().index.values)
    valset[col] = set(lgb_val[col].value_counts().index.values)

21500000
0
21500000
55000000


# In val not in train

app - 5%  
device - 16.3%  
os - 14.1%  
channel - 0.0%  

In [55]:
for col in [ 'app', 'device', 'os', 'channel','ip']:
    print(col)
    pro = len(valset[col] - trainset[col]) / len(valset[col])
    print(pro)
    print('-------')

app
0.06005221932114883
-------
device
0.1453055141579732
-------
os
0.1411042944785276
-------
channel
0.011428571428571429
-------
ip
0.48494051346274264
-------


In [60]:
valset[col] - trainset[col]

set()

In [61]:
len(valset['app'])

401

In [62]:
len(trainset['app'])

706